In [35]:
import os
import openai
import sys
import numpy as np
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

# Load Pdf Document 

In [5]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./Seg Models.pdf")
pages = loader.load()

In [10]:
print(
    "Num of Pages",len(pages)  
    ,"Page 1 content",pages[1].page_content
)

Num of Pages 14 Page 1 contetn batch_norm
:
if
specified
as
True,
all
convolutional
layers
will
be
configured
as
stacks
of
"Conv2D-BN-Activation"
.
Type:
Bolean
Ex:
True
,
False
stack_num_down
:
number
of
convolutional
layers
per
downsampling
level.
Type:
int
Ex:
1
,
min
:1
max:500
stack_num_up
:
number
of
convolutional
layers
(after
concatenation)
per
upsampling
level.
Type:
int
Ex:
1
,
min
:1
max:500
pool
:
the
configuration
of
downsampling
(encoding)
blocks.
Type:list
Ex:
False
:
downsampling
with
a
convolutional
layer
(2-by-2
convolution
kernels
with
2
strides;
optional
batch
normalization
and
activation).
True
or
'max'
downsampling
with
a
max-pooling
layer.
'ave'
downsampling
with
a
average-pooling
layer
unpool
:
the
configuration
of
upsampling
(decoding)
blocks.
Type:list
Ex:
False
:
upsampling
with
a
transpose
convolutional
layer
(2-by-2
convolution
kernels
with
2
strides;
optional
batch
normalization
and
activation)
True
or
'bilinear'
upsampling
with
bilinear.
'nearest'
upsampl

# Split the Documnet 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [14]:
text_splitter = CharacterTextSplitter(
    separator="Model:", ## split on that 
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)
docs = text_splitter.split_documents(pages)

#   Create Embbeding 

In [18]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embedding = OpenAIEmbeddings()

In [20]:
persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory)

In [26]:
vectordb._collection.count() == len(docs)

True

In [30]:
vectordb._collection.get(include=['embeddings'])


{'ids': ['00d21681-a188-11ee-8305-f89e9475a728',
  '00d21682-a188-11ee-a799-f89e9475a728',
  '00d21683-a188-11ee-a0fd-f89e9475a728',
  '00d21684-a188-11ee-a7da-f89e9475a728',
  '00d21685-a188-11ee-acc1-f89e9475a728',
  '00d21686-a188-11ee-bbdc-f89e9475a728',
  '00d21687-a188-11ee-a91b-f89e9475a728',
  '00d21688-a188-11ee-b62c-f89e9475a728',
  '00d21689-a188-11ee-bcaa-f89e9475a728',
  '00d2168a-a188-11ee-ae4a-f89e9475a728',
  '00d2168b-a188-11ee-9b35-f89e9475a728',
  '00d2168c-a188-11ee-9ca2-f89e9475a728',
  '00d2168d-a188-11ee-98e1-f89e9475a728',
  '00d2168e-a188-11ee-ba6b-f89e9475a728'],
 'embeddings': [[-0.046577573078118446,
   -0.011556233342810246,
   0.016177257611697086,
   0.001547381872419035,
   -0.0070821419746814876,
   0.004224306461294858,
   -0.001981750840018179,
   -0.011401954169543293,
   -0.01999749999289109,
   -0.036556780538855964,
   0.02599234148219503,
   0.025816023491401458,
   -0.020790934676752474,
   -0.01724986368941173,
   0.019527316200581357,
   -0.00

In [40]:
np.array(vectordb._collection.get(include=['embeddings'])['embeddings'][9])


array([-0.03466195,  0.01118703,  0.03012466, ..., -0.01764338,
        0.00787704, -0.04150508])

# Similarty search

In [43]:
# will get the moset neares question depending on the embeeding 
question = "Give me Dtatils about UNET model PARAMTERS IN ARK"
docs = vectordb.similarity_search(question,k=1)

In [44]:
docs

[Document(page_content="Notes:\n●\nFor\nparameters\nlacking\ndefined\nmin/max\nvalues\nor\nlist\nsize\nconstraints,\ntheir\nconfiguration\ndepends\non\nthe\nuser's\ndiscretion\nand\nthe\ncapabilities\nof\nthe\ndevice\nhosting\nthe\nARK\ncontainer\n.\n●\nWhile\nvalues\nmust\nexceed\none,\nthere's\nno\nset\nmaximum,\nallowing\ncustomization\nbased\non\nthe\ndevice's\ncomputational\ncapacity\nand\nspecific\nuser\nrequirements.", metadata={'page': 13, 'source': './Seg Models.pdf'})]

# Retrival (RAG)

In [45]:
from langchain.llms import OpenAI

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [46]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

NameError: name 'OpenAI' is not defined